In [1]:
from train import reward_fn  
CKPT = "mistral-grpo-demo"

[2025-05-23 18:13:15,475] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
from datasets import load_dataset
val_ds = load_dataset("trl-lib/tldr", split="validation[:1%]")

row = val_ds[0]
for key, value in row.items():
    print(f"{key}>>\n {value}")
    print("--\n\n")
val_ds.shape

prompt>>
 SUBREDDIT: r/AskReddit

TITLE: How do you get someone out of your head?

POST: Hi,
I'm 22, and I have been with my girlfriend for 5 years now. We recently moved together. We've always loved each other intensely.

Problem, I recently started to have feelings for an other person (a friend). This person has had a boyfriend for now 3 years, and has absolutely no ideas. Those feelings were so strong, it was hard to hide them. After 2 months of me being distant and really sad, my girlfriend forced me to say what was bothering me. I'm not a good liar, and now she knows.

We decided to give us a week alone, I went to my parents. 

Now, I'm completely lost. I keep on thinking about this person, and I hate that. I would like for those feelings to go away, to leave me alone. But I can't.  

What do I do? It's been 3 months now, and I'm just desperate.

TL;DR:
--


completion>>
  long relationship; fell in love with an other person; admitted it; would like it to disappear, though it does

(64, 2)

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tok = AutoTokenizer.from_pretrained(CKPT, padding_side="left")
model = AutoModelForCausalLM.from_pretrained(
    CKPT,
    torch_dtype=torch.float16,          # A100-friendly
    device_map="auto"                  # splits across the 4 GPUs
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [25]:
import textwrap

prompt = row["prompt"]

print("Prompt:")
print(prompt)
print("\n\n")
print("Summary:")

inputs   = tok(prompt, return_tensors="pt").to(model.device)
outputs  = model.generate(**inputs, max_new_tokens=40, do_sample=False, temperature=0.7)

final_text = tok.decode(outputs[0], skip_special_tokens=True)
final_text = final_text.replace(prompt, "")
final_text

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt:
SUBREDDIT: r/AskReddit

TITLE: How do you get someone out of your head?

POST: Hi,
I'm 22, and I have been with my girlfriend for 5 years now. We recently moved together. We've always loved each other intensely.

Problem, I recently started to have feelings for an other person (a friend). This person has had a boyfriend for now 3 years, and has absolutely no ideas. Those feelings were so strong, it was hard to hide them. After 2 months of me being distant and really sad, my girlfriend forced me to say what was bothering me. I'm not a good liar, and now she knows.

We decided to give us a week alone, I went to my parents. 

Now, I'm completely lost. I keep on thinking about this person, and I hate that. I would like for those feelings to go away, to leave me alone. But I can't.  

What do I do? It's been 3 months now, and I'm just desperate.

TL;DR:



Summary:


'💔💔💔💔💔💔💔💔💔💔'

In [26]:
len(final_text)

10